# Lambda 1: Extracting Entities for Form Enrollment

In [19]:
import boto3
import json
import os
import sys
import base64
import time
import tzlocal
import re
from botocore.exceptions import ClientError
import logging

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

bedrock = boto3.client(
    service_name='bedrock', 
    region_name='us-east-1'
)

##Loading JSON to read data
file_path = "Conversation_2_14_formatted.json"
# Open the JSON file for reading
with open(file_path, 'r') as file:
    # Parse the JSON file
    data = json.load(file)

##Processing data to fetch role and content
def data_preprocessing(transcription):
    convo = ""
    for i in range(len(transcription['transcriptions'])):
        convo = convo + transcription['transcriptions'][i]['ParticipantRole'] + ": " + transcription['transcriptions'][i]['Content']
        convo += "\n"
    return convo

final_transcript = data_preprocessing(data)


###Below is the LLM model to extract entities

#Defining function to connect to Bedrock LLM
def load_claude2(bedrock_runtime , prompt , temp , top_p,top_k):
    try:
        body = {
            "prompt": prompt,
            "temperature": temp,
            "top_p": top_p,
            "top_k":top_k,
            "max_tokens_to_sample": 1000
            }

        response = bedrock_runtime.invoke_model(
            modelId="anthropic.claude-v2", body=json.dumps(body), accept="application/json", contentType="application/json"
                 )
        
        response_body = json.loads(response["body"].read())
        completion = response_body.get("completion")

        return completion

    except ClientError:
        logging.error("Couldn't invoke Llama 2")
        raise

#Defining LLM function for the prompt generator for entity extraction  
entities = "name of patient, status of insurance, insurance number, demographic details"
def enrollment_prompt_generator(conversation,entities):
    prompt_claude = """Human: \"""" + conversation + """\"

    The above is a transcript between a call center agent and an insurance subscriber or patient. Identify and extract key entities such as \"""" + entities + """\" from the transcript. Include only the information present.

    Output the results as a structured JSON containing only the extracted fields.

    Assistant:
    """

    return prompt_claude

# Lambda handler to intgerate with AWS
def lambda_handler1(data):
    final_transcript = data_preprocessing(data)
    prompt_enrollment = enrollment_prompt_generator(final_transcript,entities)
    #print(prompt_enrollment)
    enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)

    pattern = re.compile(r"```json\n([\s\S]*?)\n```", re.MULTILINE)
    match = pattern.search(enrollment_data)
    if match:
        json_content = match.group(1)
        #print(json_content)
    else:
        print("No JSON content found.")
    #enrollment_json_object = json.loads(enrollment_data)
    return json_content#{"statusCode": 200,"body": json.dumps(enrollment_json_object)}

d1 = lambda_handler1(data)
print(d1)

{
  "patient_name": "Jordan Andrew Smith",
  "patient_gender": "Male",
  "patient_dob": "March 14, 1989", 
  "patient_phone": "555-123-4567",
  "patient_email": "jordan.smith@email.com",
  "insurance_provider": "HealthCare Plus",
  "insurance_policy_number": "HP123456789",
  "insurance_group_number": "987654"
}
{
  "patient_name": "Jordan Andrew Smith",
  "patient_gender": "Male",
  "patient_dob": "March 14, 1989", 
  "patient_phone": "555-123-4567",
  "patient_email": "jordan.smith@email.com",
  "insurance_provider": "HealthCare Plus",
  "insurance_policy_number": "HP123456789",
  "insurance_group_number": "987654"
}


In [20]:
d1

'{\n  "patient_name": "Jordan Andrew Smith",\n  "patient_gender": "Male",\n  "patient_dob": "March 14, 1989", \n  "patient_phone": "555-123-4567",\n  "patient_email": "jordan.smith@email.com",\n  "insurance_provider": "HealthCare Plus",\n  "insurance_policy_number": "HP123456789",\n  "insurance_group_number": "987654"\n}'

# Testing

In [25]:
import logging

In [6]:
data2 = {'stream': 'TRANSCRIPTION',
 'contactId': 'aa621db9-934b-462c-bc7a-85f2e01c4c9f',
 'transcriptions': [{'ParticipantId': 'CUSTOMER',
   'ParticipantRole': 'CUSTOMER',
   'Content': "Hello I'm Mark",
   'BeginOffsetMillis': 1257,
   'EndOffsetMillis': 9697,
   'Id': 'fb29489e-d06b-48b1-8e0b-519f17c4a68e',
   'Sentiment': 'NEUTRAL',
   'IssuesDetected': []}]}

d2 = lambda_handler1(data2)
print(d2)

 {
  "name": "Mark"
}


In [7]:
data3 = {'stream': 'TRANSCRIPTION',
 'contactId': 'aa621db9-934b-462c-bc7a-85f2e01c4c9f',
 'transcriptions': [{'ParticipantId': 'CUSTOMER',
   'ParticipantRole': 'CUSTOMER',
   'Content': "Thank you, Jordan. May I ask which gender you identify with? This information helps us to tailor our care to your specific needs, but please know that providing this information is entirely optional",
   'BeginOffsetMillis': 1257,
   'EndOffsetMillis': 9697,
   'Id': 'fb29489e-d06b-48b1-8e0b-519f17c4a68e',
   'Sentiment': 'NEUTRAL',
   'IssuesDetected': []}]}

d3 = lambda_handler1(data3)
print(d3)

 {
  "agent_name": "Jordan"
}


In [8]:
data4 = {'stream': 'TRANSCRIPTION',
 'contactId': 'aa621db9-934b-462c-bc7a-85f2e01c4c9f',
 'transcriptions': [{'ParticipantId': 'CUSTOMER',
   'ParticipantRole': 'CUSTOMER',
   'Content': "Hello",
   'BeginOffsetMillis': 1257,
   'EndOffsetMillis': 9697,
   'Id': 'fb29489e-d06b-48b1-8e0b-519f17c4a68e',
   'Sentiment': 'NEUTRAL',
   'IssuesDetected': []}]}

d4 = lambda_handler1(data4)
print(d4)

 {
  "name": null,
  "insurance_status": null, 
  "insurance_number": null,
  "demographics": null
}


In [ ]:
data2 = {'stream': 'TRANSCRIPTION',
 'contactId': 'aa621db9-934b-462c-bc7a-85f2e01c4c9f',
 'transcriptions': [{'ParticipantId': 'CUSTOMER',
   'ParticipantRole': 'CUSTOMER',
   'Content': "Hello I'm Mark",
   'BeginOffsetMillis': 1257,
   'EndOffsetMillis': 9697,
   'Id': 'fb29489e-d06b-48b1-8e0b-519f17c4a68e',
   'Sentiment': 'NEUTRAL',
   'IssuesDetected': []}]}

